## QR 1: Decomposition

### Problem 1:

In [37]:
import numpy as np
from scipy import linalg as la

def qrdecomp(A):
    dimA = np.shape(A)
    m = dimA[0]
    n = dimA[1]
    Q = np.copy(A)
    R = np.zeros((n, n))
    for i in range(n):
        R[i,i] = la.norm(Q[:,i])
        Q[:,i] /= R[i,i]
        for j in range(i+1, n):
            R[i,j] = Q[:,j].T @ Q[:,i]
            Q[:,j] = Q[:,j] - R[i,j] * Q[:,i]
    
    return Q, R

In [38]:
# Testing qrdecomp function

A = np.random.random((6,4))
Q,R = qrdecomp(A)
print(A.shape, Q.shape, R.shape)

np.allclose(np.triu(R), R) # True
np.allclose(np.dot(Q.T, Q), np.identity(4)) # True
np.allclose(np.dot(Q,R), A) #True 

(6, 4) (6, 4) (4, 4)


True

### Problem 2

In [41]:
def qr_det(A):
    return abs(qrdecomp(A)[1].diagonal().prod())

### Problem 3:

In [53]:
def lin_solve(A, b):
    n = np.size(b)
    Q, R = qrdecomp(A)
    y = Q.T @ b
    
    x = np.zeros_like(b)
    
    x[-1] = 1. / R[-1, -1] * y[-1]
    for i in range(n-2, -1, -1):
        x[i] = 1. / A[i, i] * (y[i] - np.sum(A[i, i+1:] * x[i+1:]))
    
    return x

### Problem 4:

In [82]:
def householder(A):
    sign = lambda x: 1 if x >= 0 else -1
    
    m,n = np.shape(A)
    R = np.copy(A)
    Q = np.identity(m)
    for k in range(0, n):
        u = np.copy(R[k:,k])
        u[0] += sign(u[0])*la.norm(u)
        u /= la.norm(u)
        R[k:,k:] -= 2*np.outer(u, u.T@R[k:,k:])
        Q[k:,:] -= 2*np.outer(u,u.T@Q[k:,:])
    
    return Q.T, R

In [83]:
A = np.random.random((5, 3))
Q, R = householder(A)
print(A.shape, Q.shape, R.shape)
np.allclose(Q.dot(R), A)

(5, 3) (5, 5) (5, 3)


True

### Problem 5:

In [85]:
def hessenberg(A):
    sign = lambda x: 1 if x >= 0 else -1
    
    m,n = np.shape(A)
    H = np.copy(A)
    Q = np.identity(m)
    for k in range(0, n-2):
        u = np.copy(H[k+1:,k])
        u[0] += sign(u[0]) * la.norm(u)
        u /= la.norm(u)
        H[k+1:,k:] -= 2*np.outer(u, u.T@H[k+1:,k:])
        H[:,k+1:] -= 2*np.outer(H[:,k+1:]@u, u.T)
        Q[k+1:,:] -= 2*np.outer(u, u.T@Q[k+1:,:])
    
    return H, Q.T

In [87]:
A = np.random.random((8,8))
H, Q = hessenberg(A)

np.allclose(np.triu(H, -1), H)
np.allclose(np.dot(np.dot(Q,H), Q.T), A)

True